In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

#Deep learning
#Import Package
import keras
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential,Input,Model
from keras.layers import Dense,Flatten,Dropout,MaxPooling2D,Conv2D,GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint,EarlyStopping,TensorBoard,CSVLogger,ReduceLROnPlateau
from keras.models import model_from_json

import time

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Callbacks Function
def Fun_Callbacks(model_name):
    path_callback = '/content/'+ str(model_name)

    try:
      path_callback = '/content/'+ str(model_name)      
      os.mkdir(path_callback) 
    except:
      print('Folder Existed Before, Continuing to Train \n')

    best_model_weights = path_callback+'/base_best.hdf5'
    checkpoint = ModelCheckpoint(
        best_model_weights,
        monitor='val_accuracy',
        verbose=1,
        save_best_only=True,
        mode='max',        
        period=1    
    )

    earlystop = EarlyStopping(
        monitor='val_loss',
        min_delta=0.001,
        patience=10,
        verbose=1,
        mode='auto'
    )

    tensorboard = TensorBoard(
        log_dir = path_callback+'/logs',
        histogram_freq=0,
        batch_size=16,
        write_graph=True,
        write_grads=True,
        write_images=False,
    )

    csvlogger = CSVLogger(
        filename= path_callback+'/training_csv.log',
        separator = ",",
        append = False
    )

    reduce = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=40,
        verbose=1, 
        mode='auto',
        cooldown=1 
    )

    callbacks = [checkpoint,tensorboard,csvlogger,reduce]
    return callbacks

#Save Model
def SaveModel(model,model_name):
  path_callback = '/content/'+ str(model_name)
  model_json = model.to_json()
  with open(path_callback+"/model.json", "w") as json_file:
      json_file.write(model_json)

  model.save_weights(path_callback+"/"+model_name+".h5")
  print("Model save to directory: ", model_name, 'folder')

#Convert Time
def Convert(s): 
    s = s % (24 * 3600) 
    h = s // 3600
    s %= 3600
    m = s // 60
    s %= 60
      
    return "%d:%d:%d" % (h, m, s) 

#History Visualization
def PlotHistory(hist):
  plt.figure(figsize=(20,10))
  plt.subplot(1, 2, 1)
  plt.title('Loss History When Its Training')
  plt.xlabel('Epoch', fontsize=16)
  plt.ylabel('Loss', fontsize=16)
  plt.plot(hist.history['loss'], label='Training Loss')
  plt.plot(hist.history['val_loss'], label='Validation Loss')
  plt.legend(loc='upper right')

  plt.subplot(1, 2, 2)
  plt.title('Accuracy History When Its Training')
  plt.xlabel('Epoch', fontsize=16)
  plt.ylabel('Accuracy', fontsize=16)
  plt.plot(hist.history['accuracy'], label='Training Accuracy')
  plt.plot(hist.history['val_accuracy'], label='Validation Accuracy')
  plt.legend(loc='lower right')
  plt.show()

#Running Model Function
def Run(model, epoch_val, model_name, image_train, image_val):
  start = time.time()
  print("-------------------------")
  print("Training Start, Good Luck")
  print("-------------------------")

  history = model.fit_generator(
      generator = image_train, steps_per_epoch=1, epochs=epoch_val, verbose=1,
      validation_data=image_val, validation_steps=1,
      callbacks=Fun_Callbacks(model_name)
  )

  end = time.time()

  print("---------------")
  print('Time Ended, Your Model Training Has Taken Time: ',Convert(end - start),', Have a Nice Day')
  print("---------------")

  return history

def LoadCP(model,model_name):
  model.load_weights('/content/'+ str(model_name)+"/base_best.hdf5")
  print('Model Loaded From Checkpoint')
  return model

def plot_cm(y_true, y_pred, test_label, figsize=(5,5)):
    cm = confusion_matrix(y_true, y_pred)
    cm_sum = np.sum(cm, axis=1, keepdims=True)
    cm_perc = cm / cm_sum.astype(float) * 100
    annot = np.empty_like(cm).astype(str)
    nrows, ncols = cm.shape
    for i in range(nrows):
        for j in range(ncols):
            c = cm[i, j]
            p = cm_perc[i, j]
            if i == j:
                s = cm_sum[i]
                annot[i, j] = '%.1f%%\n%d/%d' % (p, c, s)
            elif c == 0:
                annot[i, j] = ''
            else:
                annot[i, j] = '%.1f%%\n%d' % (p, c)
    cm = pd.DataFrame(cm, index=np.unique(y_pred), columns= np.unique(y_true))
    cm.index = np.unique(test_label)
    cm.columns = np.unique(test_label)
    cm.index.name = 'ACTUAL'
    cm.columns.name = 'PREDICTED'
    fig, ax = plt.subplots(figsize=figsize)
    sns.heatmap(cm, cmap= "YlGnBu", annot=annot, fmt='', ax=ax)

def Test(model,image_val):
  Test_Step_Size=image_val.n//image_val.batch_size
  y_pred = model.predict_generator(image_val,steps=Test_Step_Size+1,verbose=1)
  y_pred = np.argmax(y_pred, axis=1)
  return y_pred

def ShowCFMatrix(y_pred,image_val):
  y_true = image_val.classes
  target_names = list(image_val.class_indices.keys())
  plot_cm(image_val.classes, y_pred, target_names)

  print('Classification Report')
  print(classification_report(image_val.classes, y_pred, target_names=target_names))

In [ ]:
#Read train data
df = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')
df.head()

In [ ]:
#Split data
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.3)

In [ ]:
#Image Augmentation
train_path = '../input/plant-pathology-2021-fgvc8/train_images'
test_path = '../input/plant-pathology-2021-fgvc8/test_images'

image_aug = keras.preprocessing.image.ImageDataGenerator(
    rescale = 1/255,
    
    horizontal_flip=True, 
    vertical_flip=False,
  )

#Image Train Generator
image_train = image_aug.flow_from_dataframe(train, directory = train_path,
                                              x_col = "image", y_col = "labels", shuffle = True)

#Image Validation and Test Generator
image_val = image_aug.flow_from_dataframe(test, directory = train_path,
                                        x_col = "image", y_col = "labels", shuffle = True)

In [ ]:
#Image shape
image_size_target = (299,299)
dim = (3,)
image_shape = image_size_target + dim

#Model Transfer Learning From Keras Application
from keras.layers import *
from keras.optimizers import *
from keras.applications import *

#Load Xception Base Model
base_model = Xception(weights='imagenet',include_top=False,input_shape=image_shape)
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(12, activation='softmax')(x)

KerasModel = Model(base_model.input, predictions)
#print(KerasModel.summary())

#Compiling The Model
KerasModel.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

#Run Keras Application
milestone = Run(KerasModel, 300, 'Keras Model',image_train,image_val)
PlotHistory(milestone)
SaveModel(KerasModel,'Keras Model')

print("-----------------------------------------------------------")

In [ ]:
#Evaluation Data Validation
from sklearn.metrics import confusion_matrix
y_pred = Test(KerasModel,image_val)
confusion_matrix(image_val.classes, y_pred)